In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import yfinance as yp
from sklearn.linear_model import RANSACRegressor, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

result=0
best_acc = 0;
li =  ['MSFT','DIS','ATVI','AAPL','AMZN','GOOG']
check =input("what stock are we checking out? : ")
def func(val,result,best_acc,check):
    df = yp.download(str(li[val]),interval = '1d')
    close = np.array(df['Close'])

    priceChange = []
    xlist = []

    for i in range(len(close) - 1):
        priceChange.append((close[i] - close[i+1]))
        xlist.append(i)    

    
    priceChange = np.array(priceChange)
    xlist = np.array(xlist)

    xlist = xlist.reshape(-1,1)

    newPriceChange = []
    newXlist = []
    for i in range(len(close)-1):
        if close[i] < close[i+1]:
            newPriceChange.append(1)
        else:
            newPriceChange.append(0)
        newXlist.append(i)

    newPriceChange = np.array(newPriceChange)
    newXlist = np.array(newXlist).reshape(-1,1)
    
    X_train, X_test, y_train, y_test = train_test_split(newXlist, newPriceChange,shuffle=False,random_state=1)

    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    
    df = yp.download(check,interval='1d')
    close2 = np.array(df['Close'])
    newPriceChange2=[]
    newXlist2=[]
    for i in range(len(close2)-1):
        if close2[i] < close2[i+1]:
            newPriceChange2.append(1)
        else:
            newPriceChange2.append(0)
        newXlist2.append(i)

    newPriceChange2 = np.array(newPriceChange2)
    newXlist2 = np.array(newXlist2).reshape(-1,1)

    max_acc=0
    min_acc=1
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(newXlist2, newPriceChange2,test_size=0.6,random_state=i)
        if max_acc<model.score(X_test, y_test):
            max_acc=model.score(X_test, y_test)
        if min_acc>model.score(X_test, y_test):
            min_acc=model.score(X_test, y_test)
       
    if(best_acc<max_acc):
        result = val
        best_acc=max_acc
    return result,max_acc,min_acc,best_acc


max_acc=0
min_acc=0
for i in range(len(li)):
    result,max_acc,min_acc,best_acc=func(i,result,best_acc,check)
    
########################
########################
########################

df = yp.download(li[result],interval='1d')
close = np.array(df['Close'])

priceChange = []
xlist = []

for i in range(len(close) - 1):
    priceChange.append((close[i] - close[i+1]))
    xlist.append(i)    

    
priceChange = np.array(priceChange)
xlist = np.array(xlist)

xlist = xlist.reshape(-1,1)


ransac = RANSACRegressor(base_estimator=LinearRegression(),
                         min_samples=252, max_trials=1000,
                         loss='absolute_loss', random_state=40,
                         residual_threshold=3)

ransac.fit(abs(xlist), abs(priceChange))



newPriceChange = []
newXlist = []
for i in range(len(close)-1):
    if close[i] < close[i+1]:
        newPriceChange.append(1)
    else:
        newPriceChange.append(0)
    newXlist.append(i)

newPriceChange = np.array(newPriceChange)
newXlist = np.array(newXlist).reshape(-1,1)
    
X_train, X_test, y_train, y_test = train_test_split(newXlist, newPriceChange,shuffle=False,random_state=1)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)


df = yp.download(check,interval ='1d',start="2021-04-01")
close2 = np.array(df['Close'])

priceChange = []
xlist = []

for i in range(len(close2) - 1):
    priceChange.append((close2[i] - close2[i+1]))
    xlist.append(i)    
    
ransac = RANSACRegressor(base_estimator=LinearRegression(),
                         min_samples=20, max_trials=100,
                         loss='absolute_loss', random_state=40,
                         residual_threshold=3)
priceChange = np.array(priceChange)
xlist = np.array(xlist)
xlist = xlist.reshape(-1,1)
ransac.fit(abs(xlist),abs(priceChange))

plt.figure(figsize=(8, 8))
plt.scatter(abs(xlist), abs(priceChange),c='g')
plt.scatter(abs(xlist), abs(priceChange),c='r')
line_X = np.arange(0, len(priceChange), 1)
line_y_ransac = ransac.predict(line_X[:, np.newaxis])
plt.plot(xlist, line_y_ransac, color='black', lw=2)
plt.show()

newPriceChange2=[]
newXlist2=[]
for i in range(len(close2)-1):
    if close2[i] < close2[i+1]:
        newPriceChange2.append(1)
    else:
        newPriceChange2.append(0)
    newXlist2.append(i)

newPriceChange2 = np.array(newPriceChange2)
newXlist2 = np.array(newXlist2).reshape(-1,1)

max_acc=0
min_acc=1
ransac.fit(abs(xlist), abs(priceChange))
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(newXlist2, newPriceChange2,test_size=0.6,random_state=i)
    if max_acc<model.score(X_test, y_test):
        max_acc=model.score(X_test, y_test)
    if min_acc>model.score(X_test, y_test):
        min_acc=model.score(X_test, y_test)
if model.predict([[close2.size+1]])==1:
    prediction="[increase]"
else:
    prediction="[decrease]"
print("The prediction for the next day",prediction)
print("Decision tree accuracy is between",int(min_acc*100)," to ",int(max_acc*100))
print("The price should be within + or - ",ransac.predict([[close2.size+2]]))
print(confusion_matrix(y_test, model.predict(X_test)))
plt.show()

81
81
